In [1]:
import tensorflow as tf
from tensorflow.keras import backend as K


In [2]:
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.applications.densenet import DenseNet169
from keras.models import Model
from keras import layers
from keras.layers import Input, Dense
from keras.optimizers import Adam
from keras import backend as K
from keras.callbacks import ModelCheckpoint
import pandas as pd
import numpy as np
from tensorflow.keras.optimizers import legacy


In [3]:
df=pd.read_csv('MURA-v1.1/valid_image_paths.csv', names=['Valid_Image'])
names=df['Valid_Image'].values
valid_labels=[]

for i in names:
  if ('positive' in i):
    valid_labels.append('1')
  elif('negative' in i):
    valid_labels.append('0')

valid_labels = np.array(valid_labels)

df.insert(1, 'Valid_Label', valid_labels)
df.to_csv('Valid_set.csv', index=False)


In [4]:
df=pd.read_csv('MURA-v1.1/train_image_paths.csv', names=['Train_Image'])
names=df['Train_Image'].values
train_labels=[]

for i in names:
  if ('positive' in i):
    train_labels.append('1')
  elif('negative' in i):
    train_labels.append('0')

train_labels = np.array(train_labels)

df.insert(1, 'Train_Label', train_labels)
df.to_csv('Train_set.csv', index=False)


In [3]:
train_df = pd.read_csv("./Train_set.csv", dtype=str)
valid_df = pd.read_csv("./Valid_set.csv", dtype=str)

datagen = ImageDataGenerator(rescale=1./255, rotation_range=30)


In [4]:
train_generator = datagen.flow_from_dataframe(dataframe=train_df, directory=None,
                                              x_col="Train_Image", y_col="Train_Label",
                                              target_size=(224,224), class_mode="binary",
                                              batch_size=16, shuffle=True)

valid_generator = datagen.flow_from_dataframe(dataframe=valid_df, directory=None,
                                              x_col="Valid_Image", y_col="Valid_Label",
                                              target_size=(224,224), class_mode="binary",
                                              batch_size=16, shuffle=True)


Found 36808 validated image filenames belonging to 2 classes.
Found 3197 validated image filenames belonging to 2 classes.


In [5]:
dense_model = DenseNet169(include_top=True, weights='imagenet')


2023-10-10 22:35:44.027133: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2023-10-10 22:35:44.027392: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2023-10-10 22:35:44.027404: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2023-10-10 22:35:44.027928: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-10-10 22:35:44.028444: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [38]:
gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
else:
    print("No GPUs found")


In [6]:
dense_model.summary()


Model: "densenet169"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 zero_padding2d (ZeroPaddin  (None, 230, 230, 3)          0         ['input_1[0][0]']             
 g2D)                                                                                             
                                                                                                  
 conv1/conv (Conv2D)         (None, 112, 112, 64)         9408      ['zero_padding2d[0][0]']      
                                                                                                  
 conv1/bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1/conv[0][0]']

In [7]:
dense_model.layers.pop()


In [8]:
# Compile the model
predictions = Dense(1, activation='sigmoid')(dense_model.layers[-1].output)
model = Model(inputs=dense_model.input, outputs=predictions)

model.compile(optimizer = legacy.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])


In [9]:
# Number of train and validation steps
train_steps=train_generator.n//train_generator.batch_size
valid_steps=valid_generator.n//valid_generator.batch_size


In [10]:
filepath = "weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', save_best_only=True, verbose=1, mode='max')


In [11]:
import PIL
import scipy

In [12]:
train_steps

2300

In [14]:
# Fit the model
# history = model.fit(generator=train_generator,
#                     steps_per_epoch=train_steps,
#                     validation_data=valid_generator,
#                     validation_steps=valid_steps,
#                     epochs=15,
#                     callbacks=[checkpoint])
history = model.fit(x=train_generator,
                    steps_per_epoch=train_steps,
                    validation_data=valid_generator,
                    validation_steps=valid_steps,
                    epochs=5,
                    callbacks=[checkpoint])

Epoch 1/5
 158/2300 [=>............................] - ETA: 1:09:44 - loss: 0.6799 - accuracy: 0.6891

KeyboardInterrupt: 